## Final Model Testing

All models are tested on the test data. The test data contains the last year of the time series for each of the cities.
All Error metrics are calculated independently for 

- A) Frequency (Forecasting Horizon) ($f$)
- B) City ($c$)

The result are the distributions of the conditional errors regarding the objective vector $X$.

- $\textbf{E}(X | c, f)$
- $\textbf{E}(x_i | c, f) \ \forall i \in X$

In [99]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX as ARIMA
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from copy import deepcopy as copy
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as MSE
plt.rcParams["figure.figsize"] = (14, 10)
plt.rcParams["figure.dpi"] = 200
plt.style.use("seaborn");plt.style.use("classic")
from itertools import product
%config InlineBackend.figure_format='retina'

In [100]:
def impute_outliers(df, q):
    df2 = df.copy()
    extreme_outliers = (df2 > df.quantile(q))
    df2[extreme_outliers] = np.nan
    df2.fillna(method="ffill", inplace=True)  
    return df2

def detrend(data):
    X = [i for i in range(0, len(data))]
    X = np.reshape(X, (len(X), 1))
    y = data.values
    model = LinearRegression()
    model.fit(X, y)
    trend = model.predict(X)
    detrended = [y[i]-trend[i] for i in range(0, len(data))]
    return pd.DataFrame(detrended, columns = data.columns, index=data.index)
    
    

In [101]:
#load data
with open("data_w", "rb") as f:
    data_w = pickle.load(f)
    
with open("data_2w", "rb") as f:
    data_2w = pickle.load(f)
    
with open("data_3w", "rb") as f:
    data_3w = pickle.load(f)
    
with open("data_m", "rb") as f:
    data_m = pickle.load(f)
    
cities = list(data_m.keys())

#Important Analysis parameters
starting_year = "2008"
target_vector = ['create', 'modify','tag_add',
            'tag_del', 'tag_change','loc_change',
            'new_mapper']
outlier_threshold = .99
detrend_data=False

for dat in [data_w, data_2w, data_3w, data_m]:
    for c in dat:
        dat[c] = impute_outliers(dat[c][target_vector][starting_year:], outlier_threshold).dropna()
        if detrend_data:
            dat[c] = detrend(dat[c])
        

In [102]:
def pred_ARIMA(data, order=(2,1,1)):
    pred_vec = list()
    for col in data.columns:
        series = data[col].copy()
        model = ARIMA(series, order=order)
        model_fit = model.fit()
        output = model_fit.forecast()
        pred = output[0]
        pred_vec.append(pred)
    return np.array(pred_vec)

In [103]:
def expo_weights(T,alpha):
    t = np.array(list(range(1,T+1)))
    return np.flip(((1-alpha)*alpha**t / (1-alpha)**T)/((1-alpha)*alpha**t / (1-alpha)**T).sum())
def linear_weights(T):
    t = np.array(list(range(1,T+1)))
    return t / t.sum()

def pred_WMA(data, window, test_size, weights,alpha=None):
    preds, index = list(), list()
    df = data[-test_size-window:].copy()
    df_test = data[-test_size:].copy()
    for i in range(window, len(df)):
        now = df.iloc[i]
        prev = df.iloc[i-window:i]
        if weights == "exponential":
            prediction = prev.apply(lambda x : x*expo_weights(window, alpha)).sum()
        elif weights == "linear":
            prediction = prev.apply(lambda x : x*linear_weights(window)).sum()
        else:
            raise Exception("Unknown weight method.")
        date = df.index[i]
        preds.append(prediction)
        index.append(date)
    return pd.DataFrame(preds, index=index)

In [104]:
# This function aggregates the POI data, adds city dummies and shuffles the data.
# Then a model is fitted by using all of the data except for the testing periods
# Returns the Model
def global_training(frequency,model,max_lag, city_dummies):
    
    if frequency == "1w":
        data = copy(data_w)
        test_size = 52
    elif frequency == "2w":
        data = copy(data_2w)
        test_size = 26
    elif frequency == "3w":
        test_size = 17
        data = copy(data_3w)
    elif frequency == "4w":
        data = copy(data_m)
        test_size = 12
    cols = ['create', 'modify','tag_add',
            'tag_del', 'tag_change','loc_change',
            'new_mapper']

    dfs = list()
    for c in data: 
        df = data[c].copy()
        if model == "RF":
            #add additional temporal features for the RF
            for col in pd.get_dummies(df.index.month, prefix="month").columns:
                df[col] = pd.get_dummies(df.index.month, prefix="month")[col].values
            df["week_of_year"] = df.index.week
            df["day_of_month"] = df.index.day
            if city_dummies:
                df["city"] = [c for _ in range(len(df))]
        elif model == "VAR":
            df = df.diff().dropna()
        for col in cols:
            for l in range(1, max_lag+1):
                df[col+"(t-%s)" % l] = df[col].shift(l)
        df.reset_index(inplace=True)
        df.dropna(inplace=True)
        df = df[:-test_size]
        dfs.append(df)

    res = dfs[0].append(dfs[1])
    for frame in dfs[2:]:
        res = res.append(frame)
    del res["index"]
    res = res.reset_index()
    del res["index"]
    if city_dummies:
        encoded = pd.get_dummies(res["city"])
        del res["city"]
        res = res.join(encoded)
        
    #shuffle
    res = res.sample(frac=1).reset_index(drop=True)
    Y = res[cols]
    X = res[res.columns.difference(cols)]
    #print(X.shape, Y.shape)
    if model == "RF":
        Model = RandomForestRegressor(n_estimators=100,
                                       max_depth = None, n_jobs=-1,
                                        max_features = int(X.shape[1] / 3.))
    elif model == "VAR":
        Model = LinearRegression(n_jobs=-1)
        
    Model.fit(X, Y)
    
    return Model


In [105]:
# Aggregate the results into a large dataframe in order to do analysis of the conditional performance of the models
# This is a trivial task and just makes plotting easier
def aggregate(res_w, res_2w, res_3w, res_4w):
    city, frequency, target, error, model = [], [], [], [], []
    targets = ["create", "modify", "tag_add", "tag_del", "tag_change", "loc_change", "new_mapper"]
    for c in res_w:
        model_results = res_w[c]
        for var in targets:
            for i in range(len(model_results[var])):
                error.append(model_results[var].values[i])
                model.append(model_results[var].index[i])
                target.append(var)
                city.append(c)   
                frequency.append("1w")
    for c in res_2w:
        model_results = res_2w[c]
        for var in targets:     
            for i in range(len(model_results[var])):
                error.append(model_results[var].values[i])
                model.append(model_results[var].index[i])
                target.append(var)
                city.append(c)   
                frequency.append("2w")
    for c in res_3w:
        model_results = res_3w[c]
        for var in targets:
            for i in range(len(model_results[var])):
                error.append(model_results[var].values[i])
                model.append(model_results[var].index[i])
                target.append(var)
                city.append(c)   
                frequency.append("3w")
    for c in res_4w:
        model_results = res_4w[c]
        for var in targets:
            for i in range(len(model_results[var])):
                error.append(model_results[var].values[i])
                model.append(model_results[var].index[i])
                target.append(var)
                city.append(c)   
                frequency.append("4w")
                
    return pd.DataFrame({"City" : city,
                         "Target" : target, "Error" :error,
                        "Frequency" : frequency, "Model" : model})
  

In [108]:
#last *
def rolling_predictions(frequency, city):
    if frequency == "1w":
        data = data_w[city].copy()
        test_size = 52 
        SMA_window = 10
        EWMA_alpha, EWMA_window = .5, 15
        LWMA_window = 15
        max_lag_rf, max_lag_var = 4, 1
        max_lag_grf, max_lag_grf_d, max_lag_gvar = 4, 4, 2
    elif frequency == "2w":
        data = data_2w[city].copy()
        test_size = 26
        SMA_window = 6
        EWMA_alpha, EWMA_window = .5, 15
        LWMA_window = 10
        max_lag_grf, max_lag_grf_d, max_lag_gvar = 4, 4, 2
        max_lag_rf, max_lag_var = 4, 1
    elif frequency == "3w":
        test_size = 17
        data = data_3w[city].copy()
        SMA_window = 4
        EWMA_alpha, EWMA_window = .5, 15
        LWMA_window = 10
        max_lag_grf, max_lag_grf_d, max_lag_gvar = 4, 4, 2
        max_lag_rf, max_lag_var = 3, 1
    elif frequency == "4w":
        data = data_m[city].copy()
        test_size = 12
        SMA_window = 4
        EWMA_alpha, EWMA_window = .5, 15
        LWMA_window = 6
        max_lag_rf, max_lag_var = 2, 1
        max_lag_grf, max_lag_grf_d, max_lag_gvar = 2, 2, 2

    cols = target_vector
    data_rf = data.copy()
    data_grf = data.copy()
    data_grf_d = data.copy()
    data_orig = data.copy()
    data_var = data.copy().diff().dropna()
    data_gvar = data_var.copy()

    pred_BL1 = data_orig[-test_size-1:].shift(1).dropna().values
    pred_SMA = data_orig[-test_size-SMA_window:].shift(1).rolling(SMA_window).mean().dropna().values
    pred_EWMA = pred_WMA(data_orig, EWMA_window, test_size, weights="exponential",alpha=EWMA_alpha)
    pred_LWMA = pred_WMA(data_orig, LWMA_window, test_size, weights="linear",alpha=None)
    
    grf_dummy = global_training(frequency, "RF",max_lag_grf_d, city_dummies=True)
    grf = global_training(frequency, "RF",max_lag_grf, city_dummies=False)
    gvar = global_training(frequency, "VAR",max_lag_gvar, city_dummies=False)

    for col in data_rf:
        for l in range(1, max_lag_rf+1):
            data_rf[col+"(t-%s)" % l] = data_rf[col].shift(l)
    data_rf.dropna(inplace=True)
    
    for col in data_grf:
        for l in range(1, max_lag_grf+1):
            data_grf[col+"(t-%s)" % l] = data_grf[col].shift(l)
    data_grf.dropna(inplace=True)

    for col in data_grf_d:
        for l in range(1, max_lag_grf_d+1):
            data_grf_d[col+"(t-%s)" % l] = data_grf_d[col].shift(l)
    data_grf_d.dropna(inplace=True)

    for col in data_var:
        for l in range(1, max_lag_var+1):
            data_var[col+"(t-%s)" % l] = data_var[col].shift(l)
    data_var.dropna(inplace=True)

    for col in data_gvar:
        for l in range(1, max_lag_gvar+1):
            data_gvar[col+"(t-%s)" % l] = data_gvar[col].shift(l)
    data_gvar.dropna(inplace=True)

    #add additional temporal features for the RF
    for col in pd.get_dummies(data_rf.index.month, prefix="month").columns:
        data_rf[col] = pd.get_dummies(data_rf.index.month, prefix="month")[col].values
    data_rf["week_of_year"] = data_rf.index.week
    data_rf["day_of_month"] = data_rf.index.day

    for col in pd.get_dummies(data_grf.index.month, prefix="month").columns:
        data_grf[col] = pd.get_dummies(data_grf.index.month, prefix="month")[col].values
    data_grf["week_of_year"] = data_grf.index.week
    data_grf["day_of_month"] = data_grf.index.day

    for col in pd.get_dummies(data_grf_d.index.month, prefix="month").columns:
        data_grf_d[col] = pd.get_dummies(data_grf_d.index.month, prefix="month")[col].values
    data_grf_d["week_of_year"] = data_grf_d.index.week
    data_grf_d["day_of_month"] = data_grf_d.index.day

    for ci in cities:
        if ci == city:
            data_grf_d[ci] = [ 1 for i in range(len(data_grf_d))  ]
        else:
            data_grf_d[ci] = [ 0 for i in range(len(data_grf_d))  ]

    X_rf = data_rf[data_rf.columns.difference(cols)]
    X_grf = data_grf[data_grf.columns.difference(cols)]
    X_grf_d = data_grf_d[data_grf_d.columns.difference(cols)]
    X_var = data_var[data_var.columns.difference(cols)]
    X_gvar = data_gvar[data_gvar.columns.difference(cols)]
    
    y_var = data_var[cols]
    y_rf = data_rf[cols]
    preds_var, preds_rf, preds_arima = list(), list(), list()
    var_true, rf_true = list(), list()
    var_index, rf_index = list(), list()
    ## Rolling Pred with inital training
    for t in range(test_size):
        if t!= test_size-1:
            X_train_var, y_train_var = X_var[:-test_size+t], y_var[:-test_size+t]
            X_test_var, y_test_var = X_var[-test_size+t:-test_size+t+1], y_var[-test_size+t:-test_size+t+1]

            X_train_rf, y_train_rf = X_rf[:-test_size+t], y_rf[:-test_size+t]
            X_test_rf, y_test_rf = X_rf[-test_size+t:-test_size+t+1], y_rf[-test_size+t:-test_size+t+1]
        else:
            X_train_var, y_train_var = X_var[:-1], y_var[:-1]
            X_test_var, y_test_var = X_var[-1:], y_var[-1:]

            X_train_rf, y_train_rf = X_rf[:-1], y_rf[:-1]
            X_test_rf, y_test_rf = X_rf[-1:], y_rf[-1:]

        var_true.append(y_test_var.values[0])
        var_index.append(y_test_var.index[0])
        #print("Train until :", X_train_var.index[-1], " Test on : ", X_test_var.index[0])
        #print("\n - - - - - ")
        rf_true.append(y_test_rf.values[0])
        rf_index.append(y_test_rf.index[0])
        #print("Train until :",X_train_rf.index[-1], " Test on : ", X_test_rf.index[0])
        #print("\n - - - - - ")

        #Train VAR and make one step pred
        VAR = LinearRegression()
        VAR.fit(X_train_var, y_train_var)
        pred_var = VAR.predict(X_test_var)
        preds_var.append(pred_var[0])

        #Train RF and make one step pred
        RF = RandomForestRegressor(n_estimators=100,
                                   max_depth = None, n_jobs=-1, max_features = int(X_rf.shape[1] / 3.))
        RF.fit(X_train_rf, y_train_rf)
        pred_rf = RF.predict(X_test_rf)
        preds_rf.append(pred_rf[0])
        
        #Train ARIMA and make one step pred
        pred_arima = pred_ARIMA(data_orig[:-test_size+t], order=(1,1,1))
        preds_arima.append(pred_arima)

    rf_true = pd.DataFrame(rf_true, index=rf_index, columns=target_vector)
    var_true = pd.DataFrame(var_true, index=var_index, columns=target_vector)

    #preds_rf = pd.DataFrame(preds_rf, index=rf_index, columns=target_vector)
    #preds_var = pd.DataFrame(preds_var, index=var_index, columns=target_vector)
    res = {"True" : pd.DataFrame(rf_true, columns=cols, index = data_orig[-test_size:].index),
            "BL1" : pd.DataFrame(pred_BL1, columns=cols, index= data_orig[-test_size:].index),
            "SMA" : pd.DataFrame(pred_SMA, columns=cols, index= data_orig[-test_size:].index), 
            "VAR" : pd.DataFrame(preds_var, columns=cols, index= data_orig[-test_size:].index),
            "RF" : pd.DataFrame(preds_rf, columns=cols, index= data_orig[-test_size:].index),
            "LWMA" : pd.DataFrame(pred_LWMA, columns=cols, index= data_orig[-test_size:].index),
            "EWMA" : pd.DataFrame(pred_EWMA, columns=cols, index= data_orig[-test_size:].index),
            "ARIMA" : pd.DataFrame(preds_arima, columns=cols, index= data_orig[-test_size:].index),
            "GRF_D" : pd.DataFrame(grf_dummy.predict(X_grf_d[-test_size:]),
                                   columns=cols, index= data_orig[-test_size:].index),
            "GRF" : pd.DataFrame(grf.predict(X_grf[-test_size:]),
                                 columns=cols, index= data_orig[-test_size:].index),
            "GVAR" : pd.DataFrame(gvar.predict(X_gvar[-test_size:]),
                                columns=cols, index= data_orig[-test_size:].index)}


    # Rescale predictions of models which used delta(x_t)
    for x in res:
        if x in ["VAR", "GVAR"]:
            #rescale using x_t = delta(x_t) + x_{t-1}
            init_loc = data_orig.index.get_loc(res[x].index[0]) - 1
            init_val = data_orig.iloc[init_loc]
            res[x].iloc[0] = res[x].iloc[0] + init_val
            for i in range(1, len(res[x])):
                res[x].iloc[i] = res[x].iloc[i] + data_orig.iloc[init_loc+i]
    return res

In [109]:
from tqdm import tqdm

In [ ]:
# Save results frequency and city wise
res_dicts = []
for freq in tqdm(["1w", "2w", "3w", "4w"]):
    d = {}
    for city in tqdm(cities):
        res = rolling_predictions(frequency=freq, city=city)
        RMSE_rf = np.sqrt(((res["True"] - res["RF"])**2).mean())
        RMSE_grf = np.sqrt(((res["True"] - res["GRF"])**2).mean())
        RMSE_grf_dummy = np.sqrt(((res["True"] - res["GRF_D"])**2).mean())
        RMSE_gvar = np.sqrt(((res["True"] - res["GVAR"])**2).mean())
        RMSE_var = np.sqrt(((res["True"] - res["VAR"])**2).mean())
        RMSE_bl1 = np.sqrt(((res["True"] - res["BL1"])**2).mean())
        RMSE_SMA = np.sqrt(((res["True"] - res["SMA"])**2).mean())
        RMSE_LWMA = np.sqrt(((res["True"] - res["LWMA"])**2).mean())
        RMSE_EWMA = np.sqrt(((res["True"] - res["EWMA"])**2).mean())
        df = pd.DataFrame([_.to_list() for _ in [RMSE_var, RMSE_rf, RMSE_bl1, RMSE_SMA,RMSE_LWMA,RMSE_EWMA, RMSE_gvar, RMSE_grf_dummy, RMSE_grf ]],
                                                # RMSE_grf, RMSE_grf_dummy, RMSE_gvar]],
                               columns = target_vector,
                               index=["VAR", "RF", "BL1", "SMA","LWMA", "EWMA", "GVAR", "GRF_D", "GRF"])
        df.index.name = "Model"
        #df["MVE"] = df.mean(axis=1)
        #df["SVE"] = df.sum(axis=1)
        d[city] = df
    res_dicts.append(d)
    
# Restructure
model_results_w = res_dicts[0]
model_results_2w = res_dicts[1]
model_results_3w = res_dicts[2]
model_results_m = res_dicts[3]

# Make a normalized version of the results
model_results_w_norm = { city : model_results_w[city] / model_results_w[city].loc["BL1"] for city in model_results_w  }
model_results_2w_norm = { city : model_results_2w[city] / model_results_2w[city].loc["BL1"] for city in model_results_2w  }
model_results_3w_norm = { city : model_results_3w[city] / model_results_3w[city].loc["BL1"] for city in model_results_3w  }
model_results_m_norm = { city : model_results_m[city] / model_results_m[city].loc["BL1"] for city in model_results_m  }

data = aggregate(model_results_w, model_results_2w, model_results_3w, model_results_m)
data_norm = aggregate(model_results_w_norm, model_results_2w_norm, model_results_3w_norm, model_results_m_norm)